In [1]:
from __future__ import annotations

from numpy.random._generator import Generator as NumpyGenerator

import elfpy.agents.agent as agent
import elfpy.utils.sim_utils as sim_utils
import elfpy.simulators as simulators
import elfpy.utils.outputs as output_utils
import elfpy.agents.policies.random_agent as random_agent

### Setup experiment parameters

In [2]:
config = simulators.Config()

config.title = "trade tape demo"
config.pricing_model_name = "Hyperdrive" # can be yieldspace or hyperdrive

config.num_trading_days = 10 # Number of simulated trading days
config.num_blocks_per_day = 5 # Blocks in a given day (7200 means ~12 sec per block)
config.num_position_days = 8
config.trade_fee_percent = 0.10 # fee percent collected on trades
config.redemption_fee_percent = 0.005 # 5 bps

num_agents = 4 # int specifying how many agents you want to simulate
agent_budget = 1_000_000 # max money an agent can spend
trade_chance = 2 / (config.num_trading_days * config.num_blocks_per_day) # on a given block, an agent will trade with probability `trade_chance`

config.target_fixed_apr = 0.01 # target fixed APR of the initial market after the LP
config.target_liquidity = 500_000_000 # target total liquidity of the initial market, before any trades

# Define the variable apr
config.variable_apr = [0.03]*config.num_trading_days

config.do_dataframe_states = True

config.log_level = output_utils.text_to_log_level("WARNING") # Logging level, should be in ["DEBUG", "INFO", "WARNING"]
config.log_filename = "trade_tape" # Output filename for logging

config.freeze() # type: ignore

### Setup agents

In [3]:
def get_example_agents(rng: NumpyGenerator, budget: int, new_agents: int, existing_agents: int = 0) -> list[agent.Agent]:
    """Instantiate a set of custom agents"""
    agents = []
    for address in range(existing_agents, existing_agents + new_agents):
        agent = random_agent.Policy(
            rng=rng,
            trade_chance=trade_chance,
            wallet_address=address,
            budget=budget,
        )
        agent.log_status_report()
        agents += [agent]
    return agents

### Setup simulation objects

In [4]:
# define root logging parameters
output_utils.setup_logging(log_filename=config.log_filename, log_level=config.log_level)

# get an instantiated simulator object
simulator = sim_utils.get_simulator(config)

### Run the simulation

In [5]:
# add the random agents
rnd_agents = get_example_agents(
    rng=simulator.rng,
    budget=agent_budget,
    new_agents=num_agents,
    existing_agents=1,
)
simulator.add_agents(rnd_agents)
print(f"Simulator has {len(simulator.agents)} agents")

# run the simulation
simulator.run_simulation()

Simulator has 5 agents


In [7]:
sim_trades = simulator.new_simulation_state.trade_updates
print(sim_trades.trade_action)

0     MarketAction(action_type=<MarketActionType.ADD...
1     MarketAction(action_type=<MarketActionType.OPE...
2     MarketAction(action_type=<MarketActionType.OPE...
3     MarketAction(action_type=<MarketActionType.CLO...
4     MarketAction(action_type=<MarketActionType.OPE...
5     MarketAction(action_type=<MarketActionType.OPE...
6     MarketAction(action_type=<MarketActionType.OPE...
7     MarketAction(action_type=<MarketActionType.OPE...
8     MarketAction(action_type=<MarketActionType.CLO...
9     MarketAction(action_type=<MarketActionType.OPE...
10    MarketAction(action_type=<MarketActionType.OPE...
11    MarketAction(action_type=<MarketActionType.CLO...
12    MarketAction(action_type=<MarketActionType.CLO...
13    MarketAction(action_type=<MarketActionType.CLO...
14    MarketAction(action_type=<MarketActionType.CLO...
15    MarketAction(action_type=<MarketActionType.CLO...
16    MarketAction(action_type=<MarketActionType.CLO...
17    MarketAction(action_type=<MarketActionType

In [15]:
from ape import accounts, chain, networks, Project
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

/Users/dylan/code/element/elf-simulations/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Apeworx Network setup

In [16]:
networks.parse_network_choice('ethereum:local:foundry').__enter__()
project_root = Path.cwd().parent.parent
project = Project(path=project_root)

INFO: Connecting to existing 'anvil' process.


### Generate agent accounts

In [17]:
simulator.agents[0].budget
config.target_liquidity

500000000

In [18]:
governance = accounts.test_accounts.generate_test_account()
sol_agents = [governance]
for agent_address, sim_agent in simulator.agents.items():
    # make a fake agent with its own wallet
    sol_agent = accounts.test_accounts.generate_test_account()
    print(f"{sol_agent.alias=}")
    sol_agent.alias = f"agent_{agent_address}"
    ## give init_agent enough money to initialize the pool
    #sol_agent.balance += int(sim_agent.budget)
    #sol_agents.append()

sol_agent.alias='dev_11'
sol_agent.alias='dev_12'
sol_agent.alias='dev_13'
sol_agent.alias='dev_14'
sol_agent.alias='dev_15'


### Deploy contracts

In [ ]:

base_address = init_agent.deploy(project.ERC20Mintable)
base_ERC20 = project.ERC20Mintable.at(base_address)

fixed_math_address = init_agent.deploy(project.MockFixedPointMath)
fixed_math = project.MockFixedPointMath.at(fixed_math_address)

base_ERC20.mint(base_supply, sender=init_agent)
time_stretch = fixed_math.divDown(int(1e18), t_stretch)

hyperdrive_address = init_agent.deploy(
    project.MockHyperdriveTestnet,
    base_ERC20,
    initial_rate,
    share_price,
    checkpoints,
    checkpoint_duration,
    int(time_stretch),
    (curve_fee, flat_fee, gov_fee),
    governance,
)
hyperdrive = project.MockHyperdriveTestnet.at(hyperdrive_address)

with accounts.use_sender(init_agent):
    base_ERC20.approve(hyperdrive, base_supply)
    hyperdrive.initialize(base_supply, initial_apr, init_agent, False)